In [1]:
# ------ 训练数据 ------
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

trainer = Trainer(config.load("config_spacy.yml"))
training_data = load_data('training_data.json')
interpreter = trainer.train(training_data)


/anaconda3/envs/Python3/lib/python3.6/site-packages/rasa_nlu/extractors/entity_synonyms.py:116: UserWarning: Found conflicting synonym definitions for 'aapl'. Overwriting target 'AAPL' with 'TSLA'. Check your training data and remove conflicting synonym definitions to prevent this from happening.
  repr(replacement)))
/anaconda3/envs/Python3/lib/python3.6/site-packages/rasa_nlu/extractors/entity_synonyms.py:116: UserWarning: Found conflicting synonym definitions for ''. Overwriting target 'TSLA' with 'AAPL'. Check your training data and remove conflicting synonym definitions to prevent this from happening.
  repr(replacement)))
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/anaconda3/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/Python3/lib/python3.6/s

Fitting 2 folds for each of 6 candidates, totalling 12 fits


/anaconda3/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/envs/Python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.2s finished


In [2]:
# 实体识别
import spacy

# iexfinance
from iexfinance.stocks import Stock
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import get_historical_intraday

# 数据库
import sqlite3

# json
import sys
import requests
import json

# 时间
from datetime import datetime
import time

# 绘图
import matplotlib.pyplot as plt

# 正则表达式
import re

# 随机回复句子
import random

import string

In [3]:
# ------ 状态 ------

CONFUSE = -1
INIT = 0
MAIN = 1

# stock
CRT_PRICE = 2
HIS_PRICE = 3

# weather
CITY_ASK = 4
GET_WEATHER = 5

In [4]:
# ------ 回复语句 ------

response_group = {
    # ------ 客套 ------
    "greet":["Hi! I am a chatbot. What can I do for you?",
             "Nice to meet you. I'm a chatbot and I'm ready to help you.",
            ],
    "finish":["OK. Tell me when you need more assists!",
              "Alright. I'm glad to help you!",
             ],
    "function_intro":["Currently I can help you with: \n1. Get stock information \n    1.1 Get current data \n    1.2 Get historical data \n    1.3 Analyze certain stocks \n2. Get weather information(every provience in China, seven days)"],

    
    # ------ 否定 ------
    "deny":["Fine, as you wish.\n\n{}",
            "OK, I'll deal with it.\n\n{}",
           ],
    
    
    # ------ stock ------
    "current_price":["The current price of {} is {}, and there are some news about {}:\n{}",
                     "{} has a real-time price of {}, and there are some news about {}:\n{}",
                    ], 
    "vague_historical_data":["Please specify which time of data you want to query.",
                             "Which time do you want to know?"
                            ],
    "analyze":["The Earning Per Share (TTM) of {} is currently {}."],
    
    
    # ------ weather ------
    "city_ask":["Which city do you want to know?",
                "Could you please tell the exact city?",
               ],
    "weather_continue":["Here is some weather information:\n{}",
                        "I have found some information:\n{}",
                       ],
}

def resp_sentence(intent):
    return random.choice(response_group[intent])

In [5]:
# ------ 状态机 ------

policy_rules = {
    # ------------ 客套 ------------
    (INIT, "greet"): (MAIN, resp_sentence("greet"), None),
    (MAIN, "greet"): (MAIN, resp_sentence("greet"), None),
    (MAIN, "finish"): (MAIN, resp_sentence("finish"), None),
    
    # ------------ 功能介绍 ------------
    (MAIN,"function_intro"): (MAIN, resp_sentence("function_intro"), None),

    
    
    # ------------ stock ------------
    
    # ------ 当前价格 ------
    # 获取当前价格
    (MAIN, "current_price"): (CRT_PRICE, resp_sentence("current_price"), None),
    # 多次获取当前价格
    (CRT_PRICE, "current_price"): (CRT_PRICE, resp_sentence("current_price"), None),
    # 完成，返回主菜单
    (CRT_PRICE, "finish"): (MAIN, resp_sentence("finish"), None),
   
    # ------ 历史数据 ------
    # 得到清晰的历史数据信息
    (MAIN, "clear_historical_data"): (MAIN, "Here is a figure:", None),
    # 得到模糊的历史数据信息，询问详情
    (MAIN, "vague_historical_data"): (MAIN, resp_sentence("vague_historical_data"), HIS_PRICE),
    (HIS_PRICE, "vague_historical_data"): (MAIN, resp_sentence("vague_historical_data"), HIS_PRICE),
    # 得到附加信息
    (MAIN, "add_historical_data"): (HIS_PRICE, "Here is a figure:", None),
    (HIS_PRICE, "vague_historical_data"): (HIS_PRICE, "Here is a figure:", None),
    # 完成，返回主菜单
    (HIS_PRICE, "finish"): (MAIN, resp_sentence("finish"), None),
    
    # ------ 建议 ------
    (MAIN, "analyze"): (MAIN, resp_sentence("analyze"), None),
    
    
    # ------------ weather ------------
    (MAIN, "city_ask"): (MAIN, resp_sentence("city_ask"), CITY_ASK),
    (MAIN, "weather_continue"): (CITY_ASK, resp_sentence("weather_continue"), GET_WEATHER),
    (MAIN, "deny"): (GET_WEATHER, resp_sentence("deny"), None),
}

In [6]:
# ------ 核心功能 ------

# 发送消息
def send_message(state, pending, message):
    # print("old_state: ", state, "message: ", message, "pending: ", pending)
    new_state, response, pending_state = respond(state, message)
    
    # print("new_state: ", new_state, "response: ", response, "pending_state: ", pending_state)
    
    if pending is not None:
        new_state, response, pending_state = policy_rules[pending]
    if pending_state is not None:
        pending = (pending_state, get_intent(message))
        
    return new_state, pending, response, get_intent(message)



weekday = []
city = ""

# 返回状态
def respond(state, message):
    entity = get_entity(message)

    # print("res_state: ", state, "intent: ", get_intent(message))
    
    # 如果状态错误，报错 
    try:
        new_state = policy_rules[(state, get_intent(message))][0]
        # print(new_state)
    except KeyError:
        new_state = CONFUSE
    pending_state = policy_rules[(state, get_intent(message))][2]
    
    
    # ------ 客套 ------
    # 欢迎
    if get_intent(message) == 'greet':
        response = policy_rules[(state, get_intent(message))][1]
    # 结束
    if get_intent(message) == 'finish':
        response = policy_rules[(state, get_intent(message))][1]
    
    # ------ 询问功能信息 ------ 
    if get_intent(message) == 'function_intro':
        response = policy_rules[(state, get_intent(message))][1]
        
    
    # ------ 股票 ------
    # 询问当前价格
    if get_intent(message) == 'current_price':
        response = policy_rules[(state, get_intent(message))][1].format(entity, get_current_price(entity), entity, get_news(entity))
    # 询问历史价格（信息清楚）
    if get_intent(message) == 'clear_historical_data':
        response = policy_rules[(state, get_intent(message))][1]
        generate_figure(message)
    # 询问历史价格（信息模糊）
    if get_intent(message) == 'vague_historical_data':
        response = policy_rules[(state, get_intent(message))][1]
    # 询问历史价格（附加信息）
    if get_intent(message) == 'add_historical_data':
        response = policy_rules[(state, get_intent(message))][1]
        generate_figure(message)
    # 分析 及给出TTM
    if get_intent(message) == 'analyze':
        response = policy_rules[(state, get_intent(message))][1].format(entity,get_ttmEPS(entity))
    
    
    # ------ 天气 ------
    # 问用户城市
    if get_intent(message) == 'city_ask':
        response = policy_rules[(state, get_intent(message))][1]
        global weekday 
        weekday = get_weekday(message)
    # 返回天气
    if get_intent(message) == 'weather_continue':
        response = policy_rules[(state, get_intent(message))][1].format(get_weather(weekday, message))
        global city 
        city = message
    # 否定实体
    if get_intent(message) == 'deny':
        response = policy_rules[(state, get_intent(message))][1].format(get_deny_weather(weekday, city, message))
        
    return new_state, response, pending_state

# 提取意图
def get_intent(message):
    return interpreter.parse(message)['intent']['name']

# 提取实体
def get_entity(message):
    
    # 客套 没有实体
    if interpreter.parse(message)['entities'] == []:
        return []
    
    # 询问当前价格 / 历史价格 / 建议 如果实体是公司，提取公司名
    if interpreter.parse(message)['entities'][0]['entity'] == 'company':
        return interpreter.parse(message)['entities'][0]['value']
  
    # 给附加信息 提取开始和结束时间
        return [interpreter.parse(message)['entities'][0]['value'],
                interpreter.parse(message)['entities'][1]['value']]

In [7]:
# ------ 股票信息 ------

# 获取某股票的当前价格
def get_current_price(company):
    print("Company: ", company)
    
    prices = Stock(company).get_price()
    return prices

# 获取某股票的每股利润
def get_ttmEPS(company):
    ttmEPS = Stock(company).get_key_stats()['ttmEPS']
    return ttmEPS

# 获取某股票相关新闻
def get_news(company):
    news = Stock(company).get_news()
    for i in news:
        if i['summary'] != 'No summary available.':
            return i['url']
    
# 生成历史数据折线图
def generate_figure(message):
    comprehended_data = interpreter.parse(message)
  
    for i in range(0,2):
        # 获取公司名称
        if comprehended_data['entities'][i]['entity'] == 'company':
            required_company = comprehended_data['entities'][i]['value']
        
        # 获取数据类型 open / close / high
        if comprehended_data['entities'][i]['entity'] == 'hst_data_type':
            required_type = comprehended_data['entities'][i]['value']
        
        # 默认值
        else:
            required_company = 'AAPL'
            required_type = 'close'
            
    # 对模糊的历史数据询问的补充信息
    if len(comprehended_data['entities']) <= 3:
        
        # 时间格式：2019-1-1
        time_period = [comprehended_data['entities'][0]['value'],
                       comprehended_data['entities'][1]['value']]

        time1_splited = time_period[0].split(' - ')
        time2_splited = time_period[1].split(' - ')
        
        # 开始时间
        # Do data cleaning and splitting to fit into API function
        # print(time1_splited)
        
        start_year = int(time1_splited[0])
        start_month = int(time1_splited[1])
        start_day = int(time1_splited[2])
        
        # print("Start year: ", start_year, ", Start month: ", start_month, ", Start day", start_day)
        
        # 结束时间
        end_year = int(time2_splited[0])
        end_month = int(time2_splited[1])
        end_day = int(time2_splited[2])

        start_time = datetime(start_year, start_month, start_day)
        end_time = datetime(end_year, end_month, end_day)
        
        # 生成该时间段的线形图
        hst_data = get_historical_data(required_company,start_time,end_time,output_format='pandas')

    # 完整的历史数据询问
    else:
        # 时间格式：2019-1-1
        time_period = [comprehended_data['entities'][2]['value'],
                       comprehended_data['entities'][3]['value']]

        time1_splited = time_period[0].split('-')
        time2_splited = time_period[1].split('-')

        # print(time1_splited)
        
        # 开始时间
        start_year = int(time1_splited[0])
        start_month = int(time1_splited[1])
        start_day = int(time1_splited[2])
    
        # 结束时间
        end_year = int(time2_splited[0])
        end_month = int(time2_splited[1])
        end_day = int(time2_splited[2])

        start_time = datetime(start_year, start_month, start_day)
        end_time = datetime(end_year, end_month, end_day)
        
        # 生成该时间段的线形图
        hst_data = get_historical_data(required_company,start_time,end_time,output_format='pandas')
    
    # 画图
    plot_required_type = hst_data[required_type].plot()
    fig = plot_required_type.get_figure()
    fig.savefig('result.png')

In [8]:
# ------ 天气信息 ------


# ------ 获得星期数 ------
week = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':0}

def get_weekday(message):
    # 匹配询问的星期
    weekday = re.findall("[A-Z]+[a-z]*",message)

    # 没有星期，默认为今天
    if weekday == []:
        return [0]
    
    else:
        # 今天的星期
        today = int(time.strftime("%w"))

        # api中要查找的列数
        number = []
        
        for day in weekday:
            n = week[day] - today
            if(n < 0):
                n = n + 7            
            number.append(n)
            
        return number


    
# ------ 在数据库中查省份代号（用于天气api） ------

def get_citycode(city):
    conn = sqlite3.connect('city_code.db')
    c = conn.cursor()
    
    code = ''
    
    query = "SELECT * FROM city WHERE name = '" + city + "'"
    c.execute(query)
    result =  c.fetchall()
    
    for row in result:
       code = row[0]
    
    return code



# ------ 调用api返回各省天气信息 ------
def get_weather(day_list, city):

    # 申请一个key：https://www.juhe.cn/docs/api/id/39
    weather_key = "b88d6c8e427b430cb8a240be5998eb99"
    
    # 省份编号
    code = get_citycode(city)
    
    url = "http://v.juhe.cn/weather/index?format=2&cityname=" + code + "&key=" + weather_key       
    req = requests.get(url)    
    info = dict(req.json())
    info = info['result']['future']
    # print(info)
    
    
    response = ""
    
    for number in day_list:
        newinfo = info[number]
        temperature = newinfo['temperature']
        weather = newinfo['weather']
        wind = newinfo['wind']
        week = newinfo['week']
        date = newinfo['date']
        response = response + "日期: " + date + " " + week + ", 温度: " + temperature + ", 天气: " + weather + ", 风向与风力: " + wind + "\n"
        
    return response


def get_deny_weather(day_list, city, message):
    # print("old: ", day_list)
    
    # 匹配询问的星期
    weekday = re.findall("[A-Z]+[a-z]*",message)
       
    # 今天的星期
    today = int(time.strftime("%w"))
    
    # 移除否定的星期
    for day in weekday:
        n = week[day] - today
        if(n < 0):
            n = n + 7            
        day_list.remove(n)
    
    # print("new: ", day_list)
    
    return get_weather(day_list, city)
    


In [9]:
'''
# ------ 终端调试 ------

while True:
    state = MAIN
    pending = None

    msg = input()
    print("USER: " + msg)
    print("[Intent: " + get_intent(msg) + "]")

    state, pending, final_response, message_intent = send_message(state, pending, msg)

    print("BOT: " + final_response)
    
'''

# ------ 部署到微信 ------

from wxpy import *

# Create a new Bot object
bot = Bot()

# Set target client account
my_friend = bot.friends().search('恶龙')[0]


@bot.register(my_friend, TEXT)
def auto_reply(msg):
    state = MAIN
    pending = None
    print(get_intent(msg.text))
    
    state, pending, final_response, message_intent = send_message(state, pending, msg.text)
    msg.reply(final_response)
    # Save matplotlib figure to local path and send it via chatbot
    if message_intent == 'clear_historical_data' or message_intent == 'add_historical_data':
        msg.reply_image('result.png')
    return fianl_response

bot.registered


Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 二十四桥明月夜


[<MessageConfig: 二十四桥明月夜: auto_reply (Enabled, Async)>]

greet
function_intro
city_ask
weather_continue
deny
current_price
Company:  TSLA
vague_historical_data
add_historical_data
add_historical_data
analyze
finish
